# Notebook to test SetFit performance

Check CUDA

In [1]:
import torch
if torch.cuda.is_available():    
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

Import SetFit

In [3]:
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer

KeyboardInterrupt: 

Load dataset and transform data

In [ ]:
import pandas as pd
import numpy as np

dataFrame = pd.read_pickle(r'../data/7587_corrige.pkl')

In [ ]:
subset = dataFrame[['jobTitle', 'description', 'label']].copy()

subset.reset_index(drop=True, inplace=True)
subset.replace('', np.nan, inplace=True)
subset.dropna(inplace=True)

subset['text'] = subset['jobTitle'] + ' ' + subset['description']
subset['label'] = np.where((subset["label"] < 3) | (subset["label"].isna()), 0, 1)
subset = subset[['text','label']]
subset

,text,label
2,Stagiaire ingénieur en intelligence artificiel...,1
3,Stagiaire en développement logiciel Développem...,0
4,Stagiaire en développement Web Création et évo...,0
5,Stagiaire en développement Web Portage d’une a...,0
6,Développeur Data / IA Développement d'applicat...,1
...,...,...
11281,Opérateur production Montage de transmission a...,0
11282,Opérateur production Montage de transmission a...,0
11283,Technicien réparation informatique Reparation ...,0
11284,Technicien réparation Reparation & maintenance...,0


Split between training and test set and truncate the dataset to simulate few-shot context

In [ ]:
from datasets import Dataset

subset_trunc = subset.head(100)
def split_train_test(dataset, ratio):
    test_set = dataset.sample(frac = ratio)
    train_set = dataset.drop(test_set.index)
    return train_set, test_set

train_set, test_set = split_train_test(subset_trunc, 0.2)

train_ds = Dataset.from_pandas(train_set, split="train")
test_ds = Dataset.from_pandas(test_set, split="test")

In [ ]:
from setfit import sample_dataset

train_dataset = sample_dataset(train_ds, label_column="label", num_samples=8)
test_dataset = test_ds
print(train_dataset)

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 16
})


Load a pre trained sentence transformers to be used by SetFit

In [ ]:
# Load SetFit model from Hub
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")
model.to("cuda")

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_iterations=20, # Number of text pairs to generate for contrastive learning
    num_epochs=1 # Number of epochs to use for contrastive learning
)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
C:\Users\robin\AppData\Local\Temp\ipykernel_62340\2935095161.py:6: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(


Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Training and evaluation

In [41]:
trainer.train()

***** Running training *****
  Num unique pairs = 3200
  Batch size = 16
  Num epochs = 1
  Total optimization steps = 200


Step,Training Loss


***** Running evaluation *****


How to save/load the fine tuned model

In [ ]:
metrics = trainer.evaluate()
metrics

***** Running evaluation *****


In [65]:
model._save_pretrained(r'../models/setfit') # The given folder must exist

In [66]:
saved_model = SetFitModel._from_pretrained(r'../models/setfit')

A small test on unseen data

In [71]:
preds = saved_model(["Mon rôle chez DreamQuark, est de résoudre les problématiques des différents acteurs autour de la\nbanque et assurance (Churn, upsale, cross-sale etc.) à travers des techniques de Machine\nLearning/Deep learning et analyse statistique.\n\n● Contribution à l'amélioration de Brain, la plateforme d'Auto-ML de Dreamquark, en développant de nouvelles features à l'aide du framework Pytorch, Scikit-learn, Numpy, Pandas, FastApi, Docker, Kubernetes et CircleCi\n\n● Développement d'un package Time Series avec l'intégration de module automatique de preprocessing et module de training avec des réseaux de neurone TCN (Temporal Convolutional Network)\n\n● Développement d'un moteur de data-preparation scalable à l'horizontal compatible Pandas et Dask, s'inspirant de la philosophe Pandas et scikit-learn pipeline permettant de rendre reproductible les codes jupyter en production.\n\nStack Technique :\n\nPython, Pytorch, Scikit-learn, Numpy, Docker, Kubernetes, Circleci, Dask, FastApi, Dask, Azure, Circle\nCi, Prefect, Alembic, SqlAlchemy, Postgresql'","• Utilisation de Flask et d’Elasticsearch afin de créer une API\nREST pour faire des recherches sur des régions de\nplanètes.\n\n• Conception d'une application web avec Vue.js et Quasar\nutilisant cette API, avec visualisation 3D des données."])
preds

tensor([1, 0], dtype=torch.int32)